In [ ]:
#!/usr/bin/python
# -*- coding: utf-8 -*-
import RPi.GPIO as GPIO
import pyrealsense2 as rs
import serial
import numpy as np
import time
from threading import Thread
import threading
import DataDeal265 as dd
import struct
import queue
import cv2
import numpy as np
import ipywidgets
import math
import bluetooth
from Fire import ShapeDetector
from imutils import contours
from skimage import measure

#识别开启标志
recognize_flag = 1
#火情点位置记录
fire_pos_flag = 0x00
#蓝牙模块的地址和使用的端口号以及蓝牙连接状态
bt_addr = "98:da:20:03:b6:da"
blue_port=1
#设置GPIO
GPIO.setmode(GPIO.BCM) 
GPIO.setup(6,GPIO.IN)
#基础部分,0代表基础部分，1代表发挥部分
task_component_flag = 0

CopterTakingOff =1

TargetPosition = [0.0,0.0,0.0]
#视觉距飞机中心点距离
Radius = 0.11
#紧急改点
Fire_emergency_change = 0

#任务标志位
flag_data = 0x00

#基础任务的点
cp1=[0.0,0.0,0.0]
cp2=[0.0,1.5,0.0]
cp3=[0.0,3.3,0.0]
cp4=[0.8,3.3,0.0]
cp5=[0.8,0.6,0.0]
cp6=[1.6,0.6,0.0]
cp7=[1.6,3.3,0.0]
cp8=[2.4,3.3,0.0]
cp9=[2.4,0.6,0.0]
cp10=[3.2,0.6,0.0]
cp11=[3.2,3.3,0.0]
cp12=[3.8,3.3,0.0]
cp13=[3.8,0.0,0.0]
cp14=[0.0,0.0,0.0]
#发挥任务点
cp1_1=[0.0,0.0,0.0]
cp2_1=[0.0,1.0,180.0]
cp3_1=[0.0,0.0,0.0]

# 基础路径点入队
roqu=queue.Queue()
roqu.put(cp1)#
roqu.put(cp2)
roqu.put(cp3)
roqu.put(cp4)
roqu.put(cp5)
roqu.put(cp6)#
roqu.put(cp7)
roqu.put(cp8)
roqu.put(cp9)
roqu.put(cp10)
roqu.put(cp11)
roqu.put(cp12)
roqu.put(cp13)
roqu.put(cp14)
# 发挥路径点入队
roqu_1=queue.Queue()
roqu_1.put(cp1_1)#
roqu_1.put(cp2_1)
roqu_1.put(cp3_1)

# 生成图像检测器对象
sd = ShapeDetector()


CE=0.15#坐标误差判断
#高度设置有个10厘米的误差

In [ ]:
#蓝牙初始化
def Bluetoothinit():
    global sock
    global bt_addr
    global blue_port
    global flag_data
    #等待直到蓝牙准备就绪
    sock = bluetooth.BluetoothSocket(bluetooth.RFCOMM)
    while (sock.connect_ex((bt_addr,blue_port))):
        sock = bluetooth.BluetoothSocket(bluetooth.RFCOMM)
        time.sleep(1)
    print("一键起飞发送")
    #flag_data|=(0x04)
    flag_data|=(0x01<<6)

# 字符串对比
def StrComparison(str1,str2):
    n = len(str1)
    res = []
    for x in str1:
        if x in str2:
            res.append(x)
    return (n-len(res))

#以双目视觉的初始位置为原点进行坐标变换
def coordinate_conversion(x,y,yaw):
    DIF_X = Radius*math.cos((yaw+90)/180*math.pi)
    DIF_Y = Radius*math.sin((yaw+90)/180*math.pi) - Radius
    x_trs = (x+DIF_X)*math.cos(yaw/180*math.pi)+(y+DIF_Y)*math.sin(yaw/180*math.pi)
    y_trs = (y+DIF_Y)*math.cos(yaw/180*math.pi)-(x+DIF_X)*math.sin(yaw/180*math.pi)
    return x_trs,y_trs

def Angle_cri_judg(err_yaw,target_yaw,current_yaw):
    target_yaw_angle_2 = target_yaw
    if(target_yaw-err_yaw<0):
        target_yaw = target_yaw + 360.0
    if(target_yaw<=err_yaw):
        if((current_yaw<=target_yaw+err_yaw) or (current_yaw>=target_yaw_angle_2-err_yaw)):
            return 1
    else: 
        if((current_yaw<=target_yaw+err_yaw) and (current_yaw>=target_yaw_angle_2-err_yaw)):
            return 1
    return 0



#路径点更新
def Router():
    global routeNodeIndex
    global SendTargetPos
    global CopterLanding
    global LaserArray
    global LaserDistance
    global FlightMode
    global CopterTakingOff
    global routeStartFlag
    global targetx  
    global targety
    global target_yaw
    global x_real
    global y_real
    global Circle_flag
    global circle_yaw
    global err_X
    global err_Y 
    global posZ
    global flag_data
    global pos_X
    global pos_Y
    global pos_Z
    global fire_pos_flag
    global Fire_emergency_change
    global cap
    while True:
        SendTargetPos = 1    
        current_yaw = -Euler[2]*180.0/math.pi
        if(current_yaw<0.0):
            current_yaw += 360.0 
        #紧急情况改点
        if Fire_emergency_change:
            #告诉飞控紧急情况
            flag_data |= (0x01<<4) 
            #距离小于50cm
            if math.sqrt((targetx - pos_Y)**2+(targety + pos_X)**2)<0.5: 
                #飞控亮灯
                flag_data |= 0x04
            #距离小于10
            if math.sqrt((targetx - pos_Y)**2+(targety + pos_X)**2)<0.1:     
                #0.1是飞机中心点与摄像头的误差
                targetx = pos_Y
                targety = -pos_X - 0.1
                #关闭识别
                recognize_flag = 0                
                posZ = 1.0
                #循环直到到达指定高度
                while 1:
                    if(math.sqrt((1 - pos_Z)**2)<CE):
                        break
                time.sleep(3)
                #开启舵机
                flag_data |= (0x01<<5)  
                time.sleep(1)
                Fire_emergency_change = 0
                fire_pos_flag = 0x01
                posZ = 1.8
                flag_data &=~(0x01<<4) 
                flag_data &=~(0x04)
                while 1:
                    if(math.sqrt((1.8 - pos_Z)**2)<CE):
                        break
                #返回规划路径点
                targetx = float(TargetPosition[0])
                targety = float(TargetPosition[1])
                target_yaw = float(TargetPosition[2])             

        x_real,y_real = coordinate_conversion(targetx,targety,current_yaw)
        #判断通过的坐标变换
        tarx_real,tary_real = coordinate_conversion(targetx,targety,target_yaw)
        if math.sqrt((tarx_real - pos_Y)**2+(tary_real + pos_X)**2)<CE and routeStartFlag and pos_Z<=posZ+0.2 and pos_Z>=posZ-0.2:
            #这里分为基础任务或者发挥任务
                # 打开摄像头  
            if(routeNodeIndex==2):
                cap,wid = sd.openCamera()
                #摄像头识别线程
                thread_detect = Thread(target=Find_fire, args=(cap,wid,))
                thread_detect.start() 
                #如果队列点不为空，且不是紧急情况
            if not roqu.empty() and not roqu_1.empty() and Fire_emergency_change==0:
                if task_component_flag==0:
                    TargetPosition=roqu.get()
                else:
                    TargetPosition=roqu_1.get()
                routeNodeIndex+=1
                targetx = float(TargetPosition[0])
                targety = float(TargetPosition[1])
                target_yaw = float(TargetPosition[2])
                print ("route node %d: x : %.1f , y : %.1f  " %(routeNodeIndex, targetx, targety))
                time.sleep(1)
            elif roqu.empty() or roqu_1.empty():
                # pipe.stop()
                #2SendTargetPos = 0
                CopterTakingOff = 1
                CopterLanding = 1
                routeNodeIndex= 1
                print ("Landing") 
                # pipe.stop()
                # time.sleep(1.0)  
                break
    

           

#串口通信线程
def PortCom(port):
    global pipe
    global cfg
    global SendTargetPos
    global CopterLanding
    global CopterTakingOff
    global _265Ready
    global GetOnceCmd
    global routeNodeIndex
    global routeStartFlag
    global cap_x
    global cap_y
    global cap
    while(True):
#         size=port.inWaiting()
#         if(size!=0):

        response = port.readline()
        if(response !=None):
            port.flushInput()        
            CmdStr1 = str(b'Start265\n')
            CmdStr2 = str(b'Departures\n')
            CmdStr3 = str(b'Refresh265\n')
            CMD = str(response)            
            #刷新265
            if( ( StrComparison( CMD ,CmdStr1 )<=1)  and  GetOnceCmd ==False):
                print(StrComparison( CMD ,CmdStr1 ),response,CMD)
                # Declare RealSense pipeline, encapsulating the actual device and sensors
                pipe = rs.pipeline()
#                 try:
#                     pipe.stop()
#                 except:
#                     print("Error1")
                # Build config object and request pose data
                cfg = rs.config()
                cfg.enable_stream(rs.stream.pose,rs.format.any,framerate=200)
                # 使用请求的配置开始流式传输
                pipe.start(cfg)
                dd.initData()
                SendTargetPos = 1
                CopterLanding = 0
                _265Ready=True
                GetOnceCmd =True 
                routeStartFlag = True

            elif( ( StrComparison(CMD ,CmdStr2 )<=1)  and  CopterTakingOff == 1 ):
                print(StrComparison(CMD ,CmdStr2 ),response,CMD)
                print("Get!")
                #路径规划线程
                router = Thread(target=Router)
                router.start()
                CopterTakingOff = 0
                
            elif( StrComparison(CMD ,CmdStr3 )<=1):
                _265Ready = False
                GetOnceCmd = False
                routeNodeIndex= 1
                CopterTakingOff =1
                routeStartFlag = False
                print("ReStart!")
                print( StrComparison(CMD ,CmdStr3 ),response,CMD)
                try:
                    pipe.stop()
                    time.sleep(1.0)               
                except:
                    print("Error2")  
            response = 0
            CMD = 0
                    
        time.sleep(0.02) 
#蓝牙通讯线程
def blue_communication():
    global sock
    global pos_X
    global pos_Y
    global pos_Z
    global _265Ready
    global fire_pos_flag
    print("blue_ok")
    #先连接上蓝牙
    Bluetoothinit()
    blue_buf = [0]*17
    #发送数据s
    while(True):
        if _265Ready:       
            #起始标志位
            blue_buf[0] = 0xAA
            blue_buf[1] = 0xBB
            posX_buf = struct.pack("f",pos_X) 
            blue_buf[2] = posX_buf[0]
            blue_buf[3] = posX_buf[1]
            blue_buf[4] = posX_buf[2]
            blue_buf[5] = posX_buf[3] 
            posY_buf = struct.pack("f",pos_Y)
            blue_buf[6] = posY_buf[0]
            blue_buf[7] = posY_buf[1]
            blue_buf[8] = posY_buf[2]
            blue_buf[9] = posY_buf[3]
            posZ_buf = struct.pack("f",pos_Z) 
            blue_buf[10] = posZ_buf[0]
            blue_buf[11] = posZ_buf[1]
            blue_buf[12] = posZ_buf[2]
            blue_buf[13] = posZ_buf[3]
            #小车记录坐标标志位
            blue_buf[14] = fire_pos_flag
            blue_buf[15] = 0xDD
            blue_buf[16] = 0xCC
            sock.send(bytes(blue_buf))
            #只发送一次火情坐标
            if(fire_pos_flag==0x01):
                fire_pos_flag=0x00
            time.sleep(1)
# 颜色识别
# 图像处理    
# 找火
def Find_fire(cap,wid):
    #识别到火
    global Fire_emergency_change
    global number
    global err_X
    global err_Y
    global targetx  
    global targety
    global recognize_flag
    number = 0
    while recognize_flag:
        ret,image = cap.read()
        height = image.shape[0]
        width =  image.shape[1]
        #转换成HSV空间图像
        #hsv_white = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)
        #提取白色区域
        #low_white = np.array([0, 0, 170])
        #high_white = np.array([180, 20, 255])
        #low_white = np.array([0, 0, 0])
        #high_white = np.array([180, 255, 30])        
        #mask_white = cv2.inRange(hsv_white, low_white, high_white)
        #mask_white = cv2.medianBlur(mask_white, 3)  # 去椒盐噪
        #contours_1, hierarchy_1 = cv2.findContours(mask_white, 1, cv2.CHAIN_APPROX_NONE)
        #contours_1, hierarchy_1 = cv2.findContours(mask_white, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)  # 找出边界
        
        #识别灯光亮点
        gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
        blurred = cv2.GaussianBlur(gray, (11, 11), 0)
        thresh = cv2.threshold(blurred, 185, 255, cv2.THRESH_BINARY)[1]
        thresh = cv2.erode(thresh, None, iterations=2)
        thresh = cv2.dilate(thresh, None, iterations=4)
        contours, hierarchy = cv2.findContours(thresh, 1, cv2.CHAIN_APPROX_NONE)

        #对最大的白色区域进行多边形拟合
        #if len(contours_1) > 0:
            #c_1 = max(contours_1,key=cv2.contourArea)
            #x_1, y_1, w, h = cv2.boundingRect(c_1)  # 计算点集最外面的矩形边界
            #cv2.rectangle(image, (x_1, y_1), (x_1+w, y_1+h), (0, 255, 0), 2)
            #cv2.drawContours(image, c_1, -1, (0, 255, 0), 3)  # 画出边框
            # 2 找出最大轮廓
        if len(contours) > 0:
            # 最大轮廓
            c = max(contours, key=cv2.contourArea)
            #判断最大轮廓的面积
            #调整面积来获得更好的识别效果
            if 35<cv2.contourArea(c)<250:
                M = cv2.moments(c)
                # 中心点坐标
                x = int(M['m10'] / M['m00'])
                y = int(M['m01'] / M['m00'])
                # 标出中心位置
                cv2.line(image, (x, 0), (x, 720), (0, 0, 255), 1)
                cv2.line(image, (0, y), (1280, y), (0, 0, 255), 1)
                # 画出轮廓
                cv2.drawContours(image, contours, -1, (128, 0, 128), 2)
                #判断是否在白色区域内
                #if (x >=x_1 + 10) and (x <=x_1 +w-10) and (y>y_1+10) and (y>y_1 + h-10):
                #更新误差
                number+=1
                #连续扫描到5次就代表是火源
                if(number>=5):
                    #得到飞控与摄像头中心点的误差
                    err_X = x-width/2
                    err_Y = y-height/2
                    r_x = 1.8*(err_X)/1000.0
                    r_y = 1.8*(err_Y)/1000.0
                    Fire_emergency_change=1
                    #修改目标位置
                    targetx = pos_Y + r_x
                    targety = -pos_X - r_y
            else:
                number = 0
        # 显示输出图像
        sd.showImage(thresh,image,wid)
        time.sleep(0.02)
#颜色识别线程,返回识别物体与摄像头中心的位置偏差
if __name__ == '__main__':
    global routeNodeIndex # 路由节点索引
    global SendTargetPos  #发送目标位置
    global CopterLanding  # 降落 
    global LaserArray     # 雷达
    global LaserDistance  # 激光器距离
    global FlightMode     #飞行模式
    global pipe
    global _265Ready      # 265准备就绪
    global GetOnceCmd     # 获取一次命令
    global CheckSum       # 校验和
    global targetx        # 目标x
    global targety        # 目标y
    global dataBuf
    global target_yaw
    global x_real
    global y_real
    global pos_X
    global pos_Y
    global pos_Z
    global cap_x
    global cap_y 
    global cap
    #要发送的位置信息
    pos_X =0
    pos_Y =0 
    pos_Z =0
    x_real = 0
    y_real = 0
    posZ = 1.8
    # 串口
    port = serial.Serial(port="/dev/ttyAMA0",baudrate=115200,stopbits=1,parity=serial.PARITY_NONE,timeout=1000)
    if GPIO.input(6):
        task_component_flag = 1
        print("发挥任务")
    else:
        print("基础任务")   
    #蓝牙数据传输线程
    thread_blue = Thread(target=blue_communication, args=())
    thread_blue.start()
    #串口通信线程
    thread_Serial = Thread(target=PortCom, args=(port,))
    thread_Serial.start()
    #导入
    if task_component_flag==0:
        TargetPosition=roqu.get()
    else:
        TargetPosition=roqu_1.get()
    #目标位置
    targetx = TargetPosition[0]
    targety = TargetPosition[1]
    target_yaw = TargetPosition[2]
    
    routeNodeIndex = 1
    _265Ready =False
    GetOnceCmd =False
    CheckSum =0 
    dataBuf = [0]*65
    LaserArray =0
    LaserDistance=0.0
    FlightMode =0
    try:
        while(True):
            if _265Ready:
                # 等待t265相机的下一组帧
                frames = pipe.wait_for_frames()
                # 获取姿势帧
                pose = frames.get_pose_frame()
                if pose:   
                    # 将一些姿势数据打印到终端
                    data = pose.get_pose_data()
                    dataBuf,pos_X,pos_Y,pos_Z,Euler=dd.solveData(data)
                    if(SendTargetPos == 1):        
                        dataBuf[43] = 0x20
                        posX_buf = struct.pack("f",x_real) 
                        dataBuf[44] = posX_buf[0]
                        dataBuf[45] = posX_buf[1]
                        dataBuf[46] = posX_buf[2]
                        dataBuf[47] = posX_buf[3] 
                        posY_buf = struct.pack("f",y_real)
                        dataBuf[48] = posY_buf[0]
                        dataBuf[49] = posY_buf[1]
                        dataBuf[50] = posY_buf[2]
                        dataBuf[51] = posY_buf[3]
                        posZ_buf = struct.pack("f",posZ) 
                        dataBuf[52] = posZ_buf[0]
                        dataBuf[53] = posZ_buf[1]
                        dataBuf[54] = posZ_buf[2]
                        dataBuf[55] = posZ_buf[3]

                        dataBuf[56] = flag_data    
                        #问题：激光是搭载在stm32f4飞控上的，为什么在树莓派中传递给飞控          
                        Laser_Dis = struct.pack("f",target_yaw)
                        dataBuf[57] = Laser_Dis[0]
                        dataBuf[58] = Laser_Dis[1]
                        dataBuf[59] = Laser_Dis[2]             
                        dataBuf[60] = Laser_Dis[3]                  
                        dataBuf[61] = FlightMode
                    if CopterLanding==1:
                        dataBuf[62] = 0xA5
                    else:
                        dataBuf[62] = 0x00
                    
                    for i in range(0,62):                     
                        CheckSum =  CheckSum + dataBuf[i]
                
                    dataBuf[63] = 0xAA
                    dataBuf[64] = CheckSum & 0x00ff
                    
                    print("\rrpy_rad[0]:{:.2f},rpy_rad[1]:{:.2f},rpy_rad[2]:{:.2f} ,X:{:.2f},Y:{:.2f},Z:{:.2f} ".format(Euler[0]*57.3,Euler[1]*57.3,Euler[2]*57.3,pos_Y,-pos_X,pos_Z),end="")
                    port.write(dataBuf)
                    CheckSum = 0
#                     pipe.stop()
            else:
                dataBuf[0] = 0x55
                dataBuf[1] = 0xAA
                dataBuf[2] = 0xFF
                dataBuf[63] = 0xAA
                dataBuf[64] = 0x00
                port.write(dataBuf)
                time.sleep(0.1)
            # wid.value = cv2.imencode('.jpg',Frame)[1].tobytes()
    finally:
        pipe.stop()
        dataBuf[62] = 0xA5
        port.write(dataBuf)
        sock.close()
        recognize_flag = 0
        cap.release()
        print("some erro")